In [57]:
import pandas as pd
import plotly.express as px
from mainnet_launch.constants import (
    CACHE_TIME,
    eth_client,
    ALL_AUTOPOOLS,
    BAL_ETH,
    AUTO_ETH,
    AUTO_LRT,
    AutopoolConstants,
    WORKING_DATA_DIR
)
from datetime import datetime, timedelta, timezone

keeper_df = pd.read_csv(WORKING_DATA_DIR / "chainlink_keeper_upkeeper_df.csv", index_col=0, parse_dates=True)[:-6]
today = datetime.now(timezone.utc)
keeper_df = keeper_df[keeper_df.index >= today - timedelta(days=30)].copy()

keeper_df['gasCostInETH_with_chainlink_premium'].sum()

np.float64(39.441464331524934)

In [58]:
38.1 + 1.36

39.46

In [59]:
keeper_df['day'] = keeper_df.index.date
median_daily_gas_price = keeper_df.groupby('day')['gasPrice'].median().to_dict()
keeper_df['median_daily_gas_price'] = keeper_df['day'].map(median_daily_gas_price)
keeper_df['gas_price_to_use'] = keeper_df[['median_daily_gas_price', 'gasPrice']].min(axis=1)
keeper_df['gasCostInETH_if_min_of_median_or_actual'] = (keeper_df['full_tx_gas_used'] * keeper_df['gas_price_to_use'] ) /1e18
keeper_df[['gasCostInETH_if_min_of_median_or_actual', 'gasCostInETH_with_chainlink_premium']].sum()# keeper_df[['median_daily_gas_price', 'gasPrice']]

gasCostInETH_if_min_of_median_or_actual    23.261112
gasCostInETH_with_chainlink_premium        39.441464
dtype: float64

In [60]:
px.line(keeper_df['gas_price_to_use'])

In [61]:

# we use use the median 

In [75]:
bottom_90th_percentile_gas_price = keeper_df.groupby('day')['gasPrice'].apply(lambda x: x.quantile(0.1)).to_dict()
keeper_df['bottom_90th_percentile_gas_price'] = keeper_df['day'].map(bottom_90th_percentile_gas_price)
keeper_df['gas_price_to_use'] = keeper_df[['bottom_90th_percentile_gas_price', 'gasPrice']].min(axis=1)
keeper_df['gasCostInETH_if_min_of_90th_or_actual'] = (keeper_df['full_tx_gas_used'] * keeper_df['gas_price_to_use'] ) /1e18
keeper_df[['gasCostInETH_if_min_of_90th_or_actual', 'gasCostInETH_with_chainlink_premium']].sum()
# keeper_df[['median_daily_gas_price', 'gasPrice']]

gasCostInETH_if_min_of_90th_or_actual    16.308115
gasCostInETH_with_chainlink_premium      39.441464
dtype: float64

In [74]:
px.line(keeper_df['median_daily_gas_price'])

In [73]:
px.line(keeper_df['gas_price_to_use'])

In [78]:
min_daily_gas_price = keeper_df.groupby('day')['gasPrice'].min().to_dict()
keeper_df['min_gas_price'] = keeper_df['day'].map(min_daily_gas_price)
keeper_df['gas_price_to_use'] = keeper_df[['min_gas_price', 'gasPrice']].min(axis=1)
px.line(keeper_df['min_gas_price'])

In [65]:
keeper_df.columns


Index(['id', 'success', 'totalPayment', 'gasUsed', 'gasOverhead', 'trigger',
       'event', 'block', 'transaction_index', 'log_index', 'hash',
       'gasCostInETH', 'gasPrice', 'full_tx_gas_used',
       'gasCostInETH_with_chainlink_premium',
       'gasCostInETH_without_chainlink_overhead', 'day',
       'median_daily_gas_price', 'gas_price_to_use',
       'gasCostInETH_if_min_of_median_or_actual',
       'bottom_90th_percentile_gas_price',
       'gasCostInETH_if_min_of_90th_or_actual'],
      dtype='object')

In [66]:
keeper_df['gasCostInETH_with_chainlink_premium']

timestamp
2024-09-17 18:00:59+00:00    0.019801
2024-09-17 18:15:23+00:00    0.028298
2024-09-17 18:16:23+00:00    0.011470
2024-09-17 18:17:23+00:00    0.017235
2024-09-17 18:29:23+00:00    0.044308
                               ...   
2024-10-17 15:07:11+00:00    0.018436
2024-10-17 15:14:35+00:00    0.023889
2024-10-17 15:24:47+00:00    0.018311
2024-10-17 16:09:47+00:00    0.080220
2024-10-17 16:11:47+00:00    0.028558
Name: gasCostInETH_with_chainlink_premium, Length: 2415, dtype: float64

In [67]:
keeper_df["hour"] = keeper_df.index.hour
px.line(keeper_df.groupby("hour")["gasPrice"].median())

In [68]:
(keeper_df["gasPrice"] / 1e9).describe(percentiles=[0.01, 0.05, 0.1, 0.2, 0.8, 0.9, 0.95, 0.99])

count    2415.000000
mean       16.504947
std        11.515153
min         2.937618
1%          3.959603
5%          5.296050
10%         6.240087
20%         7.905032
50%        12.910345
80%        23.312397
90%        32.314982
95%        39.073939
99%        56.977734
max       102.394383
Name: gasPrice, dtype: float64

In [69]:
keeper_df[]

SyntaxError: invalid syntax (3466616537.py, line 1)

In [ ]:
px.keeper_df[["gasCostInETH", "gasPrice"]]

,gasCostInETH,gasPrice
timestamp,,
2024-09-15 00:58:47+00:00,0.002958,1462960315
2024-09-15 02:58:59+00:00,0.001057,1507133606
2024-09-15 03:10:23+00:00,0.004680,1650092328
2024-09-15 03:12:35+00:00,0.001142,1506129234
2024-09-15 03:59:11+00:00,0.001009,1470003356
...,...,...
2024-10-15 16:22:35+00:00,0.016449,23186678888
2024-10-15 16:25:59+00:00,0.034416,20721790720
2024-10-15 16:30:47+00:00,0.043482,28800068808


In [ ]:
keeper_df["percentile_bucket"] = pd.qcut(keeper_df["gasPrice"], q=20, labels=False)
keeper_df.groupby("percentile_bucket")["gasCostInETH_with_chainlink_premium"].sum()

percentile_bucket
0     0.283842
1     0.539666
2     0.705413
3     0.793173
4     0.809626
5     1.052322
6     1.089965
7     1.127785
8     1.346174
9     1.359552
10    1.668427
11    1.462218
12    1.612941
13    1.975713
14    2.234787
15    2.486748
16    2.830847
17    3.485499
18    3.902503
19    7.036631
Name: gasCostInETH_with_chainlink_premium, dtype: float64

In [ ]:
keeper_df["percentile_bucket"] = pd.qcut(keeper_df["gasPrice"], q=10, labels=False)
keeper_df.groupby("percentile_bucket")["gasCostInETH_with_chainlink_premium"].sum()

percentile_bucket
0     0.823508
1     1.498586
2     1.861947
3     2.217750
4     2.705727
5     3.130644
6     3.588654
7     4.721535
8     6.316347
9    10.939135
Name: gasCostInETH_with_chainlink_premium, dtype: float64

In [ ]:
keeper_df["percentile_bucket"] = pd.qcut(keeper_df["gasPrice"], q=10, labels=False)
keeper_df.groupby("percentile_bucket")["gasCostInETH_with_chainlink_premium"].sum()[::-1].cumsum()

percentile_bucket
9    10.939135
8    17.255482
7    21.977017
6    25.565671
5    28.696315
4    31.402042
3    33.619792
2    35.481740
1    36.980326
0    37.803834
Name: gasCostInETH_with_chainlink_premium, dtype: float64

In [ ]:
px.scatter(keeper_df["gasPrice"], title="gasPrice in gwei of keeper transactions")

In [ ]:
date_filter = pd.Timestamp("2024-10-12 19:51:48", tz="UTC")  # dates are inclusive
limited_df = keeper_df[keeper_df.index <= date_filter]
print(limited_df.index.min())
limited_df.groupby("id")["gasCostInETH_with_chainlink_premium"].sum()  # gas used +20% in eth

2024-09-15 00:58:47+00:00


id
1344461886831441856282597505993515040672606510446374000438363195934269203116     32.379633
84910810589923801598536031507827941923735631663622593132512932471876788938876     1.210564
Name: gasCostInETH_with_chainlink_premium, dtype: float64

In [ ]:
keeper_df.groupby("id")["gasCostInETH_with_chainlink_premium"].sum()  # gas used +20% in eth

id
1344461886831441856282597505993515040672606510446374000438363195934269203116     36.484062
84910810589923801598536031507827941923735631663622593132512932471876788938876     1.319772
Name: gasCostInETH_with_chainlink_premium, dtype: float64

In [ ]:
keeper_df.groupby("id")["gasCostInETH"].sum()  # gas used, without + 20% in ETH for chainlink

id
1344461886831441856282597505993515040672606510446374000438363195934269203116     30.403385
84910810589923801598536031507827941923735631663622593132512932471876788938876     1.099810
Name: gasCostInETH, dtype: float64

In [ ]:
calc_df = keeper_df[
    keeper_df["id"].apply(str) == "1344461886831441856282597505993515040672606510446374000438363195934269203116"
].copy()

px.bar(
    calc_df.resample("1D")["gasCostInETH_with_chainlink_premium"].sum(),
    title="gasCostInETH_with_chainlink_premium per day",
)

In [ ]:
keeper_df

,id,success,totalPayment,gasUsed,gasOverhead,trigger,event,block,transaction_index,log_index,hash,gasCostInETH,gasPrice,full_tx_gas_used,gasCostInETH_with_chainlink_premium,gasCostInETH_without_chainlink_overhead
timestamp,,,,,,,,,,,,,,,,
2024-09-15 00:58:47+00:00,1344461886831441856282597505993515040672606510...,True,762222831087618596,1936622,114066,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...",UpkeepPerformed,20752581,77,181,0x310c0339d78f2f4e9cddadc845b706851f66cac87dbf...,0.002958,1462960315,2021675,0.003549,0.002833
2024-09-15 02:58:59+00:00,1344461886831441856282597505993515040672606510...,True,265605280715502172,593842,112043,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...",UpkeepPerformed,20753179,6,56,0x9521b0f898dac9efe5402c9dd9d2e323bc7f0e166203...,0.001057,1507133606,701332,0.001268,0.000895
2024-09-15 03:10:23+00:00,1344461886831441856282597505993515040672606510...,True,1194124362138377949,2728563,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20753236,57,224,0x423357335cd1e934358f827333082091d235f3d4d742...,0.004680,1650092328,2836029,0.005616,0.004502
2024-09-15 03:12:35+00:00,1344461886831441856282597505993515040672606510...,True,287328072718183649,650542,112043,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...",UpkeepPerformed,20753247,123,363,0x3dbd82715f865540791d30f154f28c3df89853bfd877...,0.001142,1506129234,757996,0.001370,0.000980
2024-09-15 03:59:11+00:00,1344461886831441856282597505993515040672606510...,True,255210158701363207,583567,112043,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20753480,7,51,0xc8675c8842d175226d6aa5d1df20c313b205fb0bd082...,0.001009,1470003356,686233,0.001211,0.000858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-15 16:22:35+00:00,1344461886831441856282597505993515040672606510...,True,4482932079295759274,601422,112718,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20972128,20,116,0xe9fc831e2a64f5fd8a6494e4b89c2193f7a3aa0a36c5...,0.016449,23186678888,709427,0.019739,0.013945
2024-10-15 16:25:59+00:00,1344461886831441856282597505993515040672606510...,True,9488391523235963234,1551770,114066,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20972145,9,87,0xd44f4ebee7c5c9b9d40b1c6edb8b02b7e5283a2b0914...,0.034416,20721790720,1660871,0.041299,0.032155
2024-10-15 16:30:47+00:00,1344461886831441856282597505993515040672606510...,True,11977659084394532221,1401262,113392,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20972169,7,34,0x512d192cdb4eaa13697596cab5223449021cec7ae8f2...,0.043482,28800068808,1509797,0.052179,0.040356


In [ ]:
keeper_df.index.min(), keeper_df.index.max(),

(Timestamp('2024-09-15 00:58:47+0000', tz='UTC'),
 Timestamp('2024-10-15 16:43:35+0000', tz='UTC'))

In [ ]:
keeper_df.columns

Index(['id', 'success', 'totalPayment', 'gasUsed', 'gasOverhead', 'trigger',
       'event', 'block', 'transaction_index', 'log_index', 'hash',
       'gasCostInETH', 'gasPrice', 'full_tx_gas_used',
       'gasCostInETH_with_chainlink_premium',
       'gasCostInETH_without_chainlink_overhead'],
      dtype='object')

In [ ]:
k

NameError: name 'k' is not defined

In [ ]:
incentive_keeper_df = keeper_df[
    keeper_df["id"].apply(str) == "84910810589923801598536031507827941923735631663622593132512932471876788938876"
].copy()

incentive_keeper_df

,id,success,totalPayment,gasUsed,gasOverhead,trigger,event,block,transaction_index,log_index,hash,gasCostInETH,gasPrice,full_tx_gas_used,gasCostInETH_with_chainlink_premium,gasCostInETH_without_chainlink_overhead
timestamp,,,,,,,,,,,,,,,,
2024-09-15 08:05:23+00:00,8491081058992380159853603150782794192373563166...,True,386155791444647527,630321,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20754705,63,198,0xcb58f2b438c093cbc7699974864c36fb88aef666d1e7...,0.001534,2061336840,744137,0.001841,0.001299
2024-09-15 16:05:47+00:00,8491081058992380159853603150782794192373563166...,True,424564396036472021,630321,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20757100,10,64,0xc657571254d63819f7d2754bad5e30f692535d2d7613...,0.001679,2256081982,744101,0.002015,0.001422
2024-09-16 00:06:11+00:00,8491081058992380159853603150782794192373563166...,True,447082722796013319,630321,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20759495,26,41,0x3a19ad300c9a5f55c311f0c088193f26b94c5995e034...,0.001776,2386325842,744149,0.002131,0.001504
2024-09-16 08:06:35+00:00,8491081058992380159853603150782794192373563166...,True,1837073001820747079,630321,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20761885,12,83,0xd881a66cf57725e8784f19d555d1f73a8e6f79b55f9b...,0.007238,9727000062,744137,0.008686,0.006131
2024-09-16 16:06:59+00:00,8491081058992380159853603150782794192373563166...,True,2438540853349498225,630321,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20764261,106,245,0xfd0c6de3dc5811e82ceb087c2c4e53314cefdde81e61...,0.009601,12901415595,744149,0.011521,0.008132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-14 00:57:23+00:00,8491081058992380159853603150782794192373563166...,True,1697941129617419728,630361,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20960373,21,173,0xb205d005d2e87f04cc76c301cb8052d03ba3a9c96953...,0.006283,8442775227,744213,0.007540,0.005322
2024-10-14 08:58:11+00:00,8491081058992380159853603150782794192373563166...,True,2718960580596003150,630361,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20962766,98,336,0xa0257f7be9e7def80c992902f9f7dfec51267443935c...,0.010048,13501063233,744225,0.012057,0.008511
2024-10-14 16:58:35+00:00,8491081058992380159853603150782794192373563166...,True,7957728761550437103,630361,120137,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,UpkeepPerformed,20965148,27,66,0x61f768ee6089fcb8d1f65dd4b6ec3456c417d8277af9...,0.028797,38697010747,744177,0.034557,0.024393


In [ ]:
84910810589923801598536031507827941923735631663622593132512932471876788938876

84910810589923801598536031507827941923735631663622593132512932471876788938876

In [ ]:
incentive_keeper_df["hash"].value_counts()

hash
0xcb58f2b438c093cbc7699974864c36fb88aef666d1e7cc641a8266b34991ab0a    1
0xc657571254d63819f7d2754bad5e30f692535d2d76130afcb6d52043883380de    1
0x3a19ad300c9a5f55c311f0c088193f26b94c5995e0343ab3cd2395c238f0bc1d    1
0xd881a66cf57725e8784f19d555d1f73a8e6f79b55f9b0197a86a3507a9a8b650    1
0xfd0c6de3dc5811e82ceb087c2c4e53314cefdde81e612c123a4ee794d1747998    1
                                                                     ..
0xb205d005d2e87f04cc76c301cb8052d03ba3a9c96953cf66a28f5119e9c94930    1
0xa0257f7be9e7def80c992902f9f7dfec51267443935c08c761b88c31aa370635    1
0x61f768ee6089fcb8d1f65dd4b6ec3456c417d8277af925a93aec721d45ca0b3a    1
0x0dd8b8a96bf465f1465d5cb47557110fce0adbba26ab9561c93cd79a284ba983    1
0x55a431664f1518f619eb8ab86d51dbe5f2101d249f3f06f40987c5db7c77e5e2    1
Name: count, Length: 91, dtype: int64

In [ ]:
keeper_df["trigger"].value_counts()

trigger
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01>\xfd\xac\xef\x02\x19=\xb8\xe8\x95\xf3\xd1\xeb.>\x05k\xe9O\xc5E\x01\xdd\x95\x88_\xe4!\xa7!\xee\rC\x92\x06'          2
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01>\xdc\x8f\xe3\xf3\xb8\xb1`jf\x08\xeaA\xa368\xa8\xc1\x0bm\xd0\x8f\xdcjV2o\x05\x0c\xe8\xfb\x8c\xb1\x1d\x1d'           1
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01>\xdd\x167\x1d\x1e\xcb\xa6K\xed\x07xl\xcd \x13@\x8e]$>\xb9\xd9D\x8d\xac\xaa\xce0\x1c\xb5\x10\n\xa2\xd8'             1
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01>\xdd-\x03\xb6k"\t\x9e\xe4!Jt^\x7f\x1b\xa6\x98\x17\x8f|p\x97\x0b-*\xfc\x84\xc8\xcfl"\xc9\x00\x9d'                   1
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x